In [19]:
import networkx as nx
import pandas as pd
import random
import torch
import pickle
from sklearn.model_selection import train_test_split
import numpy as np

In [20]:
import torch
from torch_geometric.utils.convert import from_networkx
from pygod.detector import DOMINANT, OCGNN, GUIDE, GAE, GAAN, AnomalyDAE, CONAD
from pygod.metric import eval_average_precision, eval_roc_auc, eval_f1, eval_precision_at_k, eval_recall_at_k
from pygod.generator import gen_contextual_outlier, gen_structural_outlier
import pickle
import time
device = torch.device('cuda')

In [21]:
print(torch.cuda.memory_summary(device=None, abbreviated=False))

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |       0 B  |
|       from small pool |       0 B  |       0 B  |       0 B  |       0 B  |
|---------------------------------------------------------------------------|
| Active memory         |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |

# Load Dataset

In [22]:
unsw_labeled_path = "C:\\Users\\asus\\Documents\\nids-pcap-dataset\\unsw_parquet_used_dataset\\unsw_labeled.parquet"

In [23]:
unsw = pd.read_parquet(unsw_labeled_path)

In [24]:
unsw.info()
unsw.head(3)

<class 'pandas.core.frame.DataFrame'>
Index: 125180 entries, 1 to 490022
Data columns (total 13 columns):
 #   Column            Non-Null Count   Dtype   
---  ------            --------------   -----   
 0   source_ip         125180 non-null  object  
 1   destination_ip    125180 non-null  object  
 2   source_port       125180 non-null  object  
 3   destination_port  125180 non-null  object  
 4   info_message      125180 non-null  object  
 5   attack_category   15657 non-null   category
 6   is_malware        125180 non-null  int64   
 7   source_ip_info    125180 non-null  object  
 8   source_port_info  125180 non-null  object  
 9   dest_ip_info      125180 non-null  object  
 10  dest_port_info    125180 non-null  object  
 11  count_benign      125180 non-null  int64   
 12  count_malware     125180 non-null  int64   
dtypes: category(1), int64(3), object(9)
memory usage: 12.5+ MB


,source_ip,destination_ip,source_port,destination_port,info_message,attack_category,is_malware,source_ip_info,source_port_info,dest_ip_info,dest_port_info,count_benign,count_malware
index,,,,,,,,,,,,,
1,175.45.176.1,149.171.126.18,4657,80,GET /oKmwKoVbq HTTP/1.1,NaN,0,175.45.176.1 GET /oKmwKoVbq HTTP/1.1,4657 GET /oKmwKoVbq HTTP/1.1,149.171.126.18 GET /oKmwKoVbq HTTP/1.1,80 GET /oKmwKoVbq HTTP/1.1,1,0
2,175.45.176.3,149.171.126.18,32473,80,GET /level/15/exec/-/buffers/assigned/dump HTT...,NaN,1,175.45.176.3 GET /level/15/exec/-/buffers/assi...,32473 GET /level/15/exec/-/buffers/assigned/du...,149.171.126.18 GET /level/15/exec/-/buffers/as...,80 GET /level/15/exec/-/buffers/assigned/dump ...,1,7
6,175.45.176.0,149.171.126.17,49194,80,GET eLWfxXSPkc HTTP/1.1,NaN,0,175.45.176.0 GET eLWfxXSPkc HTTP/1.1,49194 GET eLWfxXSPkc HTTP/1.1,149.171.126.17 GET eLWfxXSPkc HTTP/1.1,80 GET eLWfxXSPkc HTTP/1.1,1,0


# Function

In [28]:
def split_train_test(df, test_size=0.3):
    train, test = train_test_split(df, test_size=test_size)
    return train, test

In [29]:
def graph_modeling_1(df):
    graph = nx.Graph()
    node_features = []
    labels = []

    for source_port_info in df["source_port_info"].unique():
        graph.add_node(source_port_info)
        info_message = df[df["source_port_info"] == source_port_info]["info_message"].iloc[0]
        label = df[df["source_port_info"] == source_port_info]["is_malware"].iloc[0]
        node_features.append([float(len(info_message))])
        labels.append(label)
        
    for (source_ip), group in df.groupby(["source_ip"]):
        for i in range(len(group) - 1):
            from_node = group.iloc[i]["source_port_info"]
            to_node = group.iloc[i+1]["source_port_info"]
            if graph.has_edge(from_node, to_node):
                graph[from_node][to_node]["weight"] += 1
            else:
                graph.add_edge(from_node, to_node, weight=1)
    return graph, node_features, labels

In [30]:
def graph_modeling_2(df):
    graph = nx.Graph()
    node_features = []
    labels = []

    for info_message in df["info_message"].unique():
        graph.add_node(info_message)
        node_features.append([float(len(info_message))])
    
    for (source_ip), group in df.groupby(["source_ip"]):
        for i in range(len(group) - 1):
            from_node = group.iloc[i]["info_message"]
            to_node = group.iloc[i+1]["info_message"]
            if graph.has_edge(from_node, to_node):
                graph[from_node][to_node]["weight"] += 1
            else:
                graph.add_edge(from_node, to_node, weight=1)
    node_features = torch.tensor(node_features)
    labels = torch.tensor(labels)
    return graph, node_features, labels

# Eksperimen 1

In [56]:
unsw['source_port'] = unsw.source_port.astype('int32')
unsw['destination_port']= unsw.destination_port.astype('int32')

In [57]:
train_df, test_df = split_train_test(unsw)

In [58]:
train_df.is_malware.value_counts()

is_malware
0    77233
1    10393
Name: count, dtype: int64

In [59]:
nD_train_df = train_df.drop_duplicates(subset=['info_message'], keep='first')
label_train = nD_train_df['is_malware'].to_numpy()
label_train = torch.tensor(label_train, dtype=torch.float)
value_counts = np.unique(label_train, return_counts=True)
print(value_counts)

(array([0., 1.], dtype=float32), array([1653, 5923], dtype=int64))


In [60]:
nD_train_df.is_malware.value_counts()

is_malware
1    5923
0    1653
Name: count, dtype: int64

In [69]:
test_df.is_malware.value_counts()

is_malware
0    33041
1      863
Name: count, dtype: int64

In [62]:
nD_test_df = test_df.drop_duplicates(subset=['info_message'], keep='first')
nD_test_df.is_malware.value_counts()
label_test = nD_test_df['is_malware'].to_numpy()
label_test = torch.tensor(label_test, dtype=torch.float)
value_counts = np.unique(label_test, return_counts=True)
print(value_counts)

(array([0., 1.], dtype=float32), array([ 821, 3060], dtype=int64))


In [67]:
train_graph, train_node_features = graph_modeling_2(train_df)

In [68]:
# number of nodes
print(train_graph.number_of_nodes())

7576


In [70]:
test_graph, test_node_features = graph_modeling_2(test_df)

### DOMINANT

In [75]:
pyG_train = from_networkx(train_graph)
pyG_train.x = train_node_features
pyG_test = from_networkx(test_graph)
pyG_test.x = test_node_features

In [78]:
dominant_model = DOMINANT(gpu=0, weight=0.02, num_layers=64, hid_dim=64, contamination=0.1, lr=0.001, verbose=3, epoch=100)

In [79]:
dominant_compile = dominant_model.fit(pyG_train, label_train)

Epoch 0000: Loss 4.8750 | AUC 0.6216 | Recall 0.7873 | Precision 0.7873 | AP 0.8504 | F1 0.7783 | Time 12.57
Epoch 0001: Loss 4.8739 | AUC 0.6213 | Recall 0.7874 | Precision 0.7874 | AP 0.8514 | F1 0.7874 | Time 12.25
Epoch 0002: Loss 4.8740 | AUC 0.6214 | Recall 0.7876 | Precision 0.7876 | AP 0.8514 | F1 0.7876 | Time 12.20
Epoch 0003: Loss 4.8726 | AUC 0.6213 | Recall 0.7874 | Precision 0.7874 | AP 0.8514 | F1 0.7874 | Time 12.20
Epoch 0004: Loss 4.8725 | AUC 0.6213 | Recall 0.7876 | Precision 0.7876 | AP 0.8514 | F1 0.7876 | Time 12.24
Epoch 0005: Loss 4.8718 | AUC 0.6213 | Recall 0.7876 | Precision 0.7876 | AP 0.8514 | F1 0.7876 | Time 12.26
Epoch 0006: Loss 4.8707 | AUC 0.6213 | Recall 0.7876 | Precision 0.7876 | AP 0.8514 | F1 0.7876 | Time 12.07
Epoch 0007: Loss 4.8701 | AUC 0.6214 | Recall 0.7876 | Precision 0.7876 | AP 0.8514 | F1 0.7876 | Time 11.98
Epoch 0008: Loss 4.8685 | AUC 0.6213 | Recall 0.7876 | Precision 0.7876 | AP 0.8514 | F1 0.7876 | Time 12.03
Epoch 0009: Loss 4.

In [80]:
dominant_ip_pred_res, dominant_ip_score_res, dominant_ip_prob_res, dominant_ip_conf_res = dominant_compile.predict(data=pyG_test, label = label_test,return_pred=True, return_score=True, return_prob=True, prob_method='linear', return_conf=True)

Test: Loss 0.0021 | 

ValueError: Found input variables with inconsistent numbers of samples: [3881, 1582]

In [ ]:
f1_score_ip = eval_f1(label_test, dominant_ip_pred_res)
print(f1_score_ip)
precision = eval_precision_at_k(label_test, dominant_ip_score_res, k=1606)
print(precision)
recall = eval_recall_at_k(label_test, dominant_ip_score_res, k=1606)
print(recall)

In [73]:
def dominant(train_graph, train_node_features, label_train, test_graph, test_node_features, label_test):
    pyG_train = from_networkx(train_graph)
    pyG_train.x = train_node_features
    pyG_test = from_networkx(test_graph)
    pyG_test.x = test_node_features

    model = DOMINANT(gpu=0, weight=0.02, num_layers=64, hid_dim=64, contamination=0.1, lr=0.001, verbose=3, epoch=100)
    dominant_compile = model.fit(pyG_train)
    dominant_ip_pred_res, dominant_ip_score_res, dominant_ip_prob_res, dominant_ip_conf_res = dominant_compile.predict(data=pyG_test, label = label_test,return_pred=True, return_score=True, return_prob=True, prob_method='linear', return_conf=True)
    f1_score_ip = eval_f1(label_test, dominant_ip_pred_res)
    precision = eval_precision_at_k(label_test, dominant_ip_score_res, k=3881)
    recall = eval_recall_at_k(label_test, dominant_ip_score_res, k=3881)
    return f1_score_ip, precision, recall

In [74]:
f1 = []
precision = []
recall = []
for i in range(3):
    f1_score, precision_score, recall_score = dominant(train_graph, train_node_features, label_train, test_graph, test_node_features, label_test)
    f1.append(f1_score)
    precision.append(precision_score)
    recall.append(recall_score)

Epoch 0000: Loss 4.8750 |  | Time 12.97
Epoch 0001: Loss 4.8732 |  | Time 12.00
Epoch 0002: Loss 4.8745 |  | Time 12.29


KeyboardInterrupt: 

# Eksperimen 2

In [31]:
train_df, test_df = split_train_test(unsw)

In [32]:
nD_train_df = train_df.drop_duplicates(subset=['source_port_info'], keep='first')

In [144]:
nD_train_df.is_malware.value_counts()

is_malware
0    10070
1     5381
Name: count, dtype: int64

In [140]:
value_counts = train_df.is_malware.value_counts()
if value_counts[0] > 10398:
    benign = train_df[train_df['is_malware'] == 0].sample(n=10398)
    malicious = train_df[train_df['is_malware'] == 1].sample(n=5699)
    train_df = pd.concat([benign, malicious])
    # nD_train_df = train_df.copy()

In [141]:
train_df.is_malware.value_counts()

is_malware
0    10398
1     5699
Name: count, dtype: int64

In [33]:
test_df.is_malware.value_counts()

is_malware
0    33097
1     4457
Name: count, dtype: int64

In [34]:
value_counts = test_df.is_malware.value_counts()
if value_counts[0] > 5321:
    df_to_lower = test_df[test_df['is_malware'] == 0].sample(n=5321)
    test_df = pd.concat([test_df[test_df['is_malware'] == 1], df_to_lower])
    # nD_test_df = test_df.copy()

In [147]:
nD_test_df = test_df.drop_duplicates(subset=['source_port_info'], keep='first')
nD_test_df.is_malware.value_counts()

is_malware
0    5229
1    4296
Name: count, dtype: int64

In [148]:
train_graph, train_node_features, label_train = graph_modeling_1(train_df)

In [151]:
test_graph, test_node_features, label_test = graph_modeling_1(test_df)

In [9]:
pickle.dump(train_graph, open('model_graph/train_graph.pkl', 'wb'))
pickle.dump(train_node_features, open('model_graph/train_node_features.pkl', 'wb'))
pickle.dump(label_train, open('model_graph/label_train.pkl', 'wb'))

NameError: name 'train_graph' is not defined

In [152]:
pickle.dump(test_graph, open('model_graph/test_graph.pkl', 'wb'))
pickle.dump(test_node_features, open('model_graph/test_node_features.pkl', 'wb'))
pickle.dump(label_test, open('model_graph/label_test.pkl', 'wb'))

In [35]:
train_graph = pickle.load(open('model_graph/train_graph.pkl', 'rb'))
label_train = pickle.load(open('model_graph/label_train.pkl', 'rb'))
train_node_features = pickle.load(open('model_graph/train_node_features.pkl', 'rb'))

In [36]:
test_graph = pickle.load(open('model_graph/test_graph.pkl', 'rb'))
label_test = pickle.load(open('model_graph/label_test.pkl', 'rb'))
test_node_features = pickle.load(open('model_graph/test_node_features.pkl', 'rb'))

In [37]:
train_graph.number_of_nodes()

15451

In [38]:
label_train

tensor([0, 0, 0,  ..., 1, 1, 1])

In [87]:
unique_values, counts = torch.unique(label_test, return_counts=True)
print(unique_values, counts)

tensor([0, 1]) tensor([5229, 4296])


In [39]:
test_graph.number_of_nodes()

9525

In [40]:
test_node_features

tensor([[22.],
        [34.],
        [27.],
        ...,
        [28.],
        [15.],
        [28.]])

## DOMINANT

In [65]:
import torchvision.models as models
import torch_geometric.nn as pyg_nn

In [101]:

def make_dominant_model(train_graph, train_node_features, 
                        label_train, test_graph, test_node_features):
    
    # train_node_features = torch.tensor(train_node_features)
    # label_train = torch.tensor(label_train)
    # test_node_features = torch.tensor(test_node_features)

    pyG_train = from_networkx(train_graph)
    pyG_train = pyG_train
    pyG_train.x = train_node_features

    pyG_test = from_networkx(test_graph)
    pyG_test = pyG_test
    pyG_test.x = test_node_features

    dominant_model = DOMINANT(gpu=0, weight=0.01, num_layers=64, hid_dim=64, backbone=pyg_nn.EdgeCNN, contamination=0.37, lr=0.001, verbose=3, epoch=25)  
    dominant_compile = dominant_model.fit(pyG_train, label_train)
    return dominant_compile, pyG_test

In [102]:
def predict_dominant(label_test, dominant_compile, pyG_test):
    
    dominant_ip_pred_res, dominant_ip_score_res = dominant_compile.predict(data=pyG_test, label = label_test, return_pred=True, return_score=True, prob_method='linear')
    
    unique_values, counts = torch.unique(dominant_ip_pred_res, return_counts=True)
    print(unique_values, counts)

    predictions = dominant_ip_pred_res.numpy()
    labels = label_test.numpy()
    TP = np.sum((labels == 1) & (predictions == 1))
    FN = np.sum((labels == 1) & (predictions == 0))
    FP = np.sum((labels == 0) & (predictions == 1))

    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    f1_score = 2 * (precision * recall) / (precision + recall)
    
    return f1_score_ip, precision, recall, f1_score

In [103]:
f1 = []
precision = []
recall = []
train_durration = []
predict_durration = []
f1_score_unv = []
for i in range(3):
    start_time = time.time()
    dominant_model, graph_test = make_dominant_model(train_graph, train_node_features, label_train, test_graph, test_node_features)
    make_model_runtime = time.time() - start_time
    train_durration.append(make_model_runtime)
    
    start_time = time.time()
    f1_score, precision_score, recall_score, f1_score_for = predict_dominant(label_test, dominant_model, graph_test)
    predict_runtime = time.time() - start_time
    predict_durration.append(predict_runtime)

    f1.append(f1_score)
    precision.append(precision_score)
    recall.append(recall_score)
    f1_score_unv.append(f1_score_for)

Epoch 0000: Loss 43.0706 | AUC 0.7866 | Recall 0.6608 | Precision 0.6608 | AP 0.6602 | F1 0.7074 | Time 4.50
Epoch 0001: Loss 40.1312 | AUC 0.7865 | Recall 0.6527 | Precision 0.6527 | AP 0.6587 | F1 0.7198 | Time 6.31
Epoch 0002: Loss 37.4453 | AUC 0.7862 | Recall 0.6534 | Precision 0.6534 | AP 0.6581 | F1 0.7203 | Time 13.44
Epoch 0003: Loss 34.9284 | AUC 0.7861 | Recall 0.6610 | Precision 0.6610 | AP 0.6594 | F1 0.7203 | Time 6.36
Epoch 0004: Loss 32.5632 | AUC 0.7861 | Recall 0.6534 | Precision 0.6534 | AP 0.6581 | F1 0.7203 | Time 13.37
Epoch 0005: Loss 30.3307 | AUC 0.7859 | Recall 0.6612 | Precision 0.6612 | AP 0.6589 | F1 0.7195 | Time 6.38
Epoch 0006: Loss 28.2153 | AUC 0.7851 | Recall 0.6595 | Precision 0.6595 | AP 0.6585 | F1 0.7204 | Time 13.34
Epoch 0007: Loss 26.2050 | AUC 0.7860 | Recall 0.6543 | Precision 0.6543 | AP 0.6574 | F1 0.7210 | Time 6.30
Epoch 0008: Loss 24.2915 | AUC 0.7860 | Recall 0.6543 | Precision 0.6543 | AP 0.6575 | F1 0.7210 | Time 13.45
Epoch 0009: Los

In [105]:
print("ini f1: ", f1)
print("ini precision: ", precision)
print("ini recall: ", recall)
print("ini f1_score_unv: ", f1_score_unv)

ini f1:  [0.5414656558695153, 0.5414656558695153, 0.5414656558695153]
ini precision:  [0.6355828220858896, 0.7032967032967034, 0.623574144486692]
ini recall:  [0.12057728119180633, 0.16387337057728119, 0.11452513966480447]
ini f1_score_unv:  [0.2027000586969282, 0.2658108363224467, 0.1935103244837758]


In [106]:
print(train_durration)
print(predict_durration)

[250.85678052902222, 315.7898392677307, 317.1439673900604]
[4.691774845123291, 5.131696939468384, 4.681374549865723]


## OCGNN

In [24]:
def predict_ocgnn(label_test, ocgnn_compile, pyG_test):
    ocgnn_ip_pred_res, ocgnn_ip_score_res, ocgnn_ip_prob_res, ocgnn_ip_conf_res = ocgnn_compile.predict(data=pyG_test, label = label_test,return_pred=True, return_score=True, return_prob=True, prob_method='linear', return_conf=True)
    precision = eval_precision_at_k(label_test, ocgnn_ip_score_res, k=9525)
    recall = eval_recall_at_k(label_test, ocgnn_ip_score_res, k=9525)
    f1_score_ip = 2*(precision*recall)/(precision+recall)
    return f1_score_ip, precision, recall

In [30]:
def make_ocgnn_model(train_graph, train_node_features, 
                        label_train, test_graph, test_node_features,
                        label_test):
    
    node_features = torch.tensor(node_features)
    labels = torch.tensor(labels)

    pyG_train = from_networkx(train_graph)
    pyG_train = pyG_train.cpu()
    pyG_train.x = train_node_features.cpu()
    label_train = label_train.cpu()

    pyG_test = from_networkx(test_graph)
    pyG_test = pyG_test
    pyG_test.x = test_node_features

    ocgnn_model = OCGNN(hid_dim=64, num_layers=64, weight_decay=1, 
                    contamination=0.37, lr=0.004, dropout=0.3, epoch=100, gpu=-1, 
                    beta=0.1, eps=0.001, verbose=3)
    ocgnn_compile = ocgnn_model.fit(pyG_train, label_train)
    return ocgnn_compile, pyG_test


In [31]:
f1_ocgnn = []
precision_ocgnn = []
recall_ocgnn = []
train_durration_ocgnn = []
predict_durration_ocgnn = []
for i in range(3):
    start_time = time.time()
    ocgnn_model, graph_test = make_ocgnn_model(train_graph, train_node_features, label_train, test_graph, test_node_features, label_test)
    make_model_runtime = time.time() - start_time
    train_durration_ocgnn.append(make_model_runtime)

    start_time = time.time()
    f1_score, precision_score, recall_score = predict_ocgnn(label_test, ocgnn_model, graph_test)
    predict_runtime_ocgnn = time.time() - start_time
    predict_durration_ocgnn.append(predict_runtime_ocgnn)
    f1_ocgnn.append(f1_score)
    precision_ocgnn.append(precision_score)
    recall_ocgnn.append(recall_score)

Epoch 0000: Loss 0.0000 | AUC 0.9421 | Recall 0.8220 | Precision 0.8220 | AP 0.9072 | F1 0.8220 | Time 2.92
Epoch 0001: Loss 0.0189 | AUC 0.4884 | Recall 0.3457 | Precision 0.3457 | AP 0.3505 | F1 0.3457 | Time 2.89
Epoch 0002: Loss 0.0020 | AUC 0.5033 | Recall 0.3563 | Precision 0.3563 | AP 0.3543 | F1 0.3563 | Time 2.88
Epoch 0003: Loss 0.0245 | AUC 0.4923 | Recall 0.3507 | Precision 0.3507 | AP 0.3489 | F1 0.3507 | Time 2.88
Epoch 0004: Loss 0.0026 | AUC 0.5019 | Recall 0.3548 | Precision 0.3548 | AP 0.3587 | F1 0.3548 | Time 2.84
Epoch 0005: Loss 0.0020 | AUC 0.5028 | Recall 0.3484 | Precision 0.3484 | AP 0.3507 | F1 0.3484 | Time 2.92
Epoch 0006: Loss 0.0020 | AUC 0.5056 | Recall 0.3460 | Precision 0.3460 | AP 0.3549 | F1 0.3460 | Time 2.94
Epoch 0007: Loss 0.0020 | AUC 0.4776 | Recall 0.3299 | Precision 0.3299 | AP 0.3370 | F1 0.3299 | Time 2.94
Epoch 0008: Loss 0.0020 | AUC 0.4735 | Recall 0.3241 | Precision 0.3241 | AP 0.3387 | F1 0.3241 | Time 2.88
Epoch 0009: Loss 0.0020 | AU

In [32]:
print(f1_ocgnn)
print(precision_ocgnn)
print(recall_ocgnn)

[tensor(0.6217), tensor(0.6217), tensor(0.6217)]
[tensor(0.4510), tensor(0.4510), tensor(0.4510)]
[tensor(1.), tensor(1.), tensor(1.)]


In [34]:
print(train_durration_ocgnn)
print(predict_durration_ocgnn)

[355.066321849823, 350.1418790817261, 337.6466917991638]
[2.354825735092163, 2.1781935691833496, 2.3047642707824707]


## GAE

In [64]:
def make_gae_model(train_graph, train_node_features, 
                        label_train, test_graph, test_node_features,
                        label_test):
    
    node_features = torch.tensor(node_features)
    labels = torch.tensor(labels)
    
    pyG_train = from_networkx(train_graph)
    pyG_train = pyG_train.cpu()
    pyG_train.x = train_node_features.cpu()
    label_train = label_train.cpu()

    pyG_test = from_networkx(test_graph)
    pyG_test = pyG_test
    pyG_test.x = test_node_features

    gae_model = GAE(hid_dim=64, num_layers=128, weight_decay=3.7,
                contamination=0.37, lr=0.001, epoch=100, gpu=-1,
                num_neigh=-1, verbose=3)
    
    gae_compile = gae_model.fit(pyG_train, label_train)
    return gae_compile, pyG_test


In [65]:
def predict_gae(label_test, gae_compile, pyG_test):
    gae_ip_pred_res, gae_ip_score_res, gae_ip_prob_res, gae_ip_conf_res = gae_compile.predict(data=pyG_test, label = label_test,return_pred=True, return_score=True, return_prob=True, prob_method='linear', return_conf=True)
    f1_score_ip = eval_f1(label_test, gae_ip_pred_res)
    precision = eval_precision_at_k(label_test, gae_ip_score_res, k=len(label_test))
    recall = eval_recall_at_k(label_test, gae_ip_score_res, k=len(label_test))
    f1_score = 2*(precision*recall)/(precision+recall)
    return f1_score_ip, precision, recall, f1_score

In [66]:
f1_gae = []
f1_gae_for = []
precision_gae = []
recall_gae = []
train_time_gae = []
test_time_gae = []
for i in range(3):
    start_time = time.time()
    gae_model, graph_test = make_gae_model(train_graph, train_node_features, label_train, test_graph, test_node_features, label_test)
    train_time_gae.append(time.time() - start_time)
    start_time = time.time()
    f1_score_ip, precision_score, recall_score, f1_score = predict_gae(label_test, gae_model, graph_test)
    test_time_gae.append(time.time() - start_time)
    f1_gae.append(f1_score_ip)
    precision_gae.append(precision_score)
    recall_gae.append(recall_score)
    f1_gae_for.append(f1_score)

Epoch 0000: Loss 115987.6484 | AUC 0.7865 | Recall 0.6501 | Precision 0.6501 | AP 0.6582 | F1 0.7300 | Time 5.41
Epoch 0001: Loss 115986.6797 | AUC 0.7869 | Recall 0.6616 | Precision 0.6616 | AP 0.6611 | F1 0.6624 | Time 5.46
Epoch 0002: Loss 115985.1797 | AUC 0.7873 | Recall 0.6620 | Precision 0.6620 | AP 0.6615 | F1 0.6625 | Time 5.49
Epoch 0003: Loss 115983.5625 | AUC 0.7874 | Recall 0.6620 | Precision 0.6620 | AP 0.6618 | F1 0.6622 | Time 5.50
Epoch 0004: Loss 115981.9375 | AUC 0.7873 | Recall 0.6618 | Precision 0.6618 | AP 0.6617 | F1 0.6621 | Time 6.03
Epoch 0005: Loss 115980.2188 | AUC 0.7873 | Recall 0.6620 | Precision 0.6620 | AP 0.6617 | F1 0.6621 | Time 5.93
Epoch 0006: Loss 115978.4375 | AUC 0.7873 | Recall 0.6618 | Precision 0.6618 | AP 0.6617 | F1 0.6621 | Time 5.62
Epoch 0007: Loss 115976.5625 | AUC 0.7874 | Recall 0.6620 | Precision 0.6620 | AP 0.6620 | F1 0.6622 | Time 5.66
Epoch 0008: Loss 115974.5781 | AUC 0.7873 | Recall 0.6620 | Precision 0.6620 | AP 0.6619 | F1 0.

In [67]:
print(f1_gae)
print(precision_gae)
print(recall_gae)
print(f1_gae_for)

[0.18174696435566, 0.18058277799555733, 0.17520357236669296]
[tensor(0.4510), tensor(0.4510), tensor(0.4510)]
[tensor(1.), tensor(1.), tensor(1.)]
[tensor(0.6217), tensor(0.6217), tensor(0.6217)]


In [71]:
print(train_time_gae)
print(test_time_gae)

[732.9163107872009, 711.4018852710724, 709.7734138965607]
[3.3277716636657715, 2.8125693798065186, 3.137454032897949]


## CONAD

In [13]:
def make_conad_model(train_graph, train_node_features, 
                        label_train, test_graph, test_node_features,
                        label_test):
    
    node_features = torch.tensor(node_features)
    labels = torch.tensor(labels)

    pyG_train = from_networkx(train_graph)
    pyG_train = pyG_train.cpu()
    pyG_train.x = train_node_features.cpu()
    label_train = label_train.cpu()

    pyG_test = from_networkx(test_graph)
    pyG_test = pyG_test
    pyG_test.x = test_node_features

    conad_model = CONAD(hid_dim=64, num_layers=64, 
                        contamination=0.37, lr=0.001, 
                        epoch=100, gpu=-1, num_neigh=-1, 
                        weight=0.5, eta=0.5, 
                        margin=0.5, verbose=3)
    conad_compile = conad_model.fit(pyG_train, label_train)

    return conad_compile, pyG_test

In [14]:
def predict_conad(label_test, conda_compile, pyG_test):
    conad_ip_pred_res, conad_ip_score_res, conad_ip_prob_res, conad_ip_conf_res = conda_compile.predict(data=pyG_test, label = label_test, return_pred=True, return_score=True, return_prob=True, prob_method='linear', return_conf=True)
    f1_score_ip = eval_f1(label_test, conad_ip_pred_res)
    precision = eval_precision_at_k(label_test, conad_ip_score_res, k=len(label_test))
    recall = eval_recall_at_k(label_test, conad_ip_score_res, k=len(label_test))
    f1_score = 2*(precision*recall)/(precision+recall)
    return f1_score_ip, precision, recall, f1_score

In [15]:
f1_conad = []
f1_conad_for = []
precision_conad = []
recall_conad = []
train_time_conad = []
test_time_conad = []
for i in range(3):
    start_time = time.time()
    conad_model, graph_test = make_conad_model(train_graph, train_node_features, label_train,
                                              test_graph, test_node_features, label_test)
    train_time_conad.append(time.time() - start_time)
    
    start_time = time.time()
    f1_score_ip, precision_score, recall_score, f1_score = predict_conad(label_test, conad_model, graph_test)
    test_time_conad.append(time.time() - start_time)
    
    f1_conad.append(f1_score_ip)
    precision_conad.append(precision_score)
    recall_conad.append(recall_score)
    f1_conad_for.append(f1_score)

Epoch 0000: Loss 19.1913 | AUC 0.7865 | Recall 0.6527 | Precision 0.6527 | AP 0.6587 | F1 0.7198 | Time 17.34
Epoch 0001: Loss 19.1892 | AUC 0.7872 | Recall 0.6597 | Precision 0.6597 | AP 0.6611 | F1 0.6597 | Time 12.43
Epoch 0002: Loss 19.1889 | AUC 0.7870 | Recall 0.6597 | Precision 0.6597 | AP 0.6612 | F1 0.6750 | Time 12.31
Epoch 0003: Loss 19.1858 | AUC 0.7875 | Recall 0.6597 | Precision 0.6597 | AP 0.6619 | F1 0.6722 | Time 12.76
Epoch 0004: Loss 19.1848 | AUC 0.7876 | Recall 0.6597 | Precision 0.6597 | AP 0.6619 | F1 0.6597 | Time 12.71
Epoch 0005: Loss 19.1805 | AUC 0.7875 | Recall 0.6597 | Precision 0.6597 | AP 0.6618 | F1 0.6597 | Time 12.49
Epoch 0006: Loss 19.1790 | AUC 0.7875 | Recall 0.6599 | Precision 0.6599 | AP 0.6620 | F1 0.6613 | Time 12.24
Epoch 0007: Loss 19.1766 | AUC 0.7872 | Recall 0.6599 | Precision 0.6599 | AP 0.6620 | F1 0.6686 | Time 12.27
Epoch 0008: Loss 19.1746 | AUC 0.7875 | Recall 0.6599 | Precision 0.6599 | AP 0.6620 | F1 0.6683 | Time 12.89
Epoch 0009

In [16]:
print(f1_conad)
print(precision_conad)
print(recall_conad)
print(f1_conad_for)

[0.5595794392523366, 0.5607181851267742, 0.5414656558695153]
[tensor(0.4510), tensor(0.4510), tensor(0.4510)]
[tensor(1.), tensor(1.), tensor(1.)]
[tensor(0.6217), tensor(0.6217), tensor(0.6217)]


In [17]:
print(train_time_conad)
print(test_time_conad)

[1306.3826067447662, 1317.8039546012878, 1331.694186449051]
[3.382333993911743, 3.0480153560638428, 3.529951810836792]


def 